Thais Lima de Sousa. October, 2016

# Hierarchical Image Segmentation based on an observation scale
## Guimarães and Najman's algorithm

In [1]:
from vpi.io import *
from vpi.filters import convolve, normalize
import numpy as np
import networkx as nx

In [2]:
# gaussian filter for image smoothing

def create_gauss_kernel(sigma):
    length = int(np.ceil(4*sigma)) + 1
    k = np.zeros((1, length))
    for i in range(length):
        k[0][i] = np.exp(-0.5*np.square(i/sigma))
    k = k / k.sum()

    return k

def smooth(img, sigma):
    if sigma < 0.1:
        return img
    kernel = create_gauss_kernel(sigma)  
    g = convolve(img, kernel)
    g = convolve(g, np.transpose(kernel))
    return g

In [57]:
# disjoint-set forest with union rank and path compression

class UF:
    def __init__(self, n):
        self.num = n
        self.parent = list(range(n))
        self.rank = [0 for i in range(n)]
        self.size = [1 for i in range(n)]
    
    def find(self, v):
        w = v
        while(w != self.parent[w]):
            w = self.parent[w]
        self.parent[v] = w
        return w
    
    def union(self, x, y):
        xR = self.find(x)
        yR = self.find(y)
        if xR == yR:
            return
        
        if self.rank[xR] > self.rank[yR]:
            self.parent[yR] = xR
            self.size[xR] += self.size[yR]
        
        else:
            self.parent[xR] = yR
            self.size[yR] += self.size[xR]
            if self.rank[xR] == self.rank[yR]:
                self.rank[yR] += 1
        self.num -= 1
        
    def sizec(self, x):
        x = self.find(x)
        return self.size[x]
    
    def num_sets(self):
        return self.num
    
    # for debugging
    
    def print_parent(self, x):
        return self.parent[x]
    
    def print_rank(self, x):
        return self.rank[x]

In [123]:
def build_hierarchies(G):
    V = len(G)
    mst = nx.minimum_spanning_edges(G)
    mst = list(mst)
    Pw = UF(V)
    Gh = nx.Graph()
    Int = [0.0]*V
    
    for (x, y, w) in mst:
        a = Pw.find(x)
        b = Pw.find(y)
        w = w['weight']
        # print("%d-%d w= %.2f. a= %d, b=%d. Int(a)= %.2f, Int(b)=%.2f" % (x,y,w,a,b, Int[a], Int[b]))
        Sa = (w - Int[a])*Pw.sizec(a)
        Sb = (w - Int[b])*Pw.sizec(b)
        if Sa >= Sb: Gh.add_edge(x, y, weight=Sa)
        else: Gh.add_edge(x, y, weight=Sb)
        Pw.union(a, b)
        Int[Pw.find(a)] = w
        
    return Gh
        

def segment_graph(G, k=0):
    V = len(G)
    u = UF(V)
    H = build_hierarchies(G)
    visited = [False]*V
    dfst = nx.dfs_tree(H, 0)
    top = nx.topological_sort(dfst)
    print(top)
    
    for x in range(V):
        if not visited[x]:
            visited[x] = True
            for y in H.neighbors(x):
                L = H.get_edge_data(x, y)['weight']
                print('(%d, %d, %.2f)'%(x, y, L))
        
                if L < k:
                    print("junta %d e %d" % (x, y))
                    u.union(x, y)
                    print("p[%d]=%d. p[%d]=%d" % (x, u.print_parent(x), y, u.print_parent(y)))            
    
    for v in range(V):
        print("%d, p=%d" % (v, u.print_parent(v)))
        
    
    
    

In [124]:
G = nx.Graph()
G.add_edge(0, 1, weight=11)
G.add_edge(1, 2, weight=5)
G.add_edge(0, 3, weight=1)
G.add_edge(1, 4, weight=9)
G.add_edge(2, 5, weight=3)
G.add_edge(3, 4, weight=1)
G.add_edge(4, 5, weight=1)

segment_graph(G, 9)

[0, 3, 4, 5, 2, 1]
(0, 3, 1.00)
junta 0 e 3
p[0]=3. p[3]=3
(1, 2, 10.00)
(2, 1, 10.00)
(2, 5, 8.00)
junta 2 e 5
p[2]=5. p[5]=5
(3, 0, 1.00)
junta 3 e 0
p[3]=3. p[0]=3
(3, 4, 1.00)
junta 3 e 4
p[3]=3. p[4]=3
(4, 3, 1.00)
junta 4 e 3
p[4]=3. p[3]=3
(4, 5, 1.00)
junta 4 e 5
p[4]=3. p[5]=5
(5, 2, 8.00)
junta 5 e 2
p[5]=5. p[2]=5
(5, 4, 1.00)
junta 5 e 4
p[5]=5. p[4]=5
0, p=3
1, p=1
2, p=5
3, p=5
4, p=5
5, p=5


In [76]:
G = nx.Graph()
G.add_edge(0, 1, weight=9)
G.add_edge(1, 2, weight=3)
G.add_edge(0, 3, weight=1)
G.add_edge(1, 4, weight=2)
G.add_edge(2, 5, weight=5)
G.add_edge(3, 4, weight=11)
G.add_edge(4, 5, weight=7)

build_hierarchies(G)

0-3 w= 1.00. a= 0, b=3. Int(a)= 0.00, Int(b)=0.00
1-4 w= 2.00. a= 1, b=4. Int(a)= 0.00, Int(b)=0.00
1-2 w= 3.00. a= 4, b=2. Int(a)= 2.00, Int(b)=0.00
2-5 w= 5.00. a= 4, b=5. Int(a)= 3.00, Int(b)=0.00
0-1 w= 9.00. a= 3, b=4. Int(a)= 1.00, Int(b)=5.00
(0, 1, 16.00)
(0, 3, 1.00)
(1, 2, 3.00)
(1, 4, 2.00)
(2, 5, 6.00)
